# CatBoost pipeline

Attention à la version de numpy : 1.26.4 ( au lieu de 2.2.2) pour que ça fonctionne ici

In [1]:
import pandas as pd 

dataframe = pd.read_csv("SBAnational.csv", low_memory=False)

D'abord effectuer les conversions, avant de supprimer des colonnes

In [2]:
import conversion_functions as conv
import math

print (f"Before : {dataframe.shape[0]} lines")

explanable_X = pd.DataFrame(dataframe)

explanable_X["State"] = explanable_X["State"].apply(lambda x : conv.get_state_code(x))
explanable_X["NAICS"] = explanable_X["NAICS"].apply(lambda x : conv.get_NAICS_data(x))

explanable_X["ApprovalFY"] = explanable_X["ApprovalFY"].apply(lambda x : conv.get_ApprovalFY_data(x)) 
mean_dataframe = explanable_X[ explanable_X["ApprovalFY"] !=50]
mean_value = mean_dataframe["ApprovalFY"].mean()
explanable_X.loc[explanable_X['ApprovalFY'] == 50, 'ApprovalFY'] = math.ceil(mean_value)

explanable_X["NewExist"] = explanable_X["NewExist"].apply(lambda x : conv.get_NewExist_data(x)) 
explanable_X["FranchiseCode"] = explanable_X["FranchiseCode"].apply(lambda x : conv.get_FranchiseCode_data(x)) 

explanable_X = explanable_X.drop("UrbanRural", axis=1)

explanable_X["RevLineCr"] = explanable_X["RevLineCr"].apply(lambda x : conv.get_RevLineCr_data(x)) 
explanable_X["LowDoc"] = explanable_X["LowDoc"].apply(lambda x : conv.get_LowDoc_data(x)) 

explanable_X["GrAppv"] = explanable_X["GrAppv"].apply(lambda x : conv.get_GrAppv_value(x)) 
mean_dataframe = explanable_X[ explanable_X["GrAppv"] !=0]
mean_value = mean_dataframe["GrAppv"].mean()
explanable_X.loc[explanable_X['GrAppv'] == 0, 'ApprovalFY'] = math.ceil(mean_value)

explanable_X = explanable_X[ explanable_X["GrAppv"] !=0] 

explanable_X = explanable_X.drop("SBA_Appv", axis = 1)

explanable_X['MIS_Status'] = explanable_X.apply(lambda row: conv.predict_MIS_Status_data(row), axis=1 )
explanable_X["MIS_Status"] = explanable_X["MIS_Status"].apply(lambda x : conv.get_MIS_Status_data(x))

print (f"After : {explanable_X.shape[0]} lines")


Before : 899164 lines
After : 899164 lines


Ne supprimer les colonnes qu'après les conversions ( qui utilisent les colonnes supprimées )

In [3]:
explanable_X = explanable_X.drop('LoanNr_ChkDgt', axis=1)
explanable_X = explanable_X.drop('Name', axis=1)
explanable_X = explanable_X.drop('City', axis=1)
explanable_X = explanable_X.drop('Zip', axis=1)
explanable_X = explanable_X.drop('Bank', axis=1)
explanable_X = explanable_X.drop('BankState', axis=1)
explanable_X = explanable_X.drop('ApprovalDate', axis=1)
explanable_X = explanable_X.drop('ChgOffDate', axis=1) # (explicit end on simulation?) : included 
explanable_X = explanable_X.drop('DisbursementDate', axis=1)  
explanable_X = explanable_X.drop('DisbursementGross', axis=1)  
explanable_X = explanable_X.drop('BalanceGross', axis=1)  # 14 valeurs seulement différentes de zéro  ?
explanable_X = explanable_X.drop('ChgOffPrinGr', axis=1)

In [4]:
explanable_X["MIS_Status"].value_counts()

MIS_Status
P I F     741345
CHGOFF    157819
Name: count, dtype: int64

In [5]:
import MIS_Status_reference as mis_ref

# change data type in purpose to use precision as scoring metrix
explanable_X["MIS_Status"] = explanable_X["MIS_Status"].apply(lambda x : mis_ref.convert_MIS_Status_to_int(x) )
explanable_X["MIS_Status"].value_counts()

MIS_Status
1    741345
0    157819
Name: count, dtype: int64

In [6]:
import numpy
import pandas as pd 

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, OrdinalEncoder, LabelEncoder, LabelBinarizer, OneHotEncoder, Binarizer, FunctionTransformer, PolynomialFeatures,MinMaxScaler

def identity_operation(X) :
    return X

preprocessor = make_pipeline(  
    make_column_transformer(
        # LoanNr_ChkDgt, Name , City : excluded

        (FunctionTransformer(identity_operation, validate=False), ["State"]), 
        # Zip , Bank , BankState : excluded

        (FunctionTransformer(identity_operation, validate=False), ["NAICS"]), #included (2 first chars )
        
        # ApprovalDate : excluded
        (StandardScaler(), ["ApprovalFY"]),
        (StandardScaler(), ["Term"]), 
        (StandardScaler(), ["NoEmp"]),
        (StandardScaler(), ["NewExist"]),
        (StandardScaler(), ["CreateJob"]),
        (StandardScaler(), ["RetainedJob"]),
        (Binarizer(), ["FranchiseCode"]),
        #Binarizer("UrbanRural", threshold=1.5),

        (FunctionTransformer(identity_operation, validate=False), ["RevLineCr"]),
        (Binarizer(), ["LowDoc"]),
        # SimpleImputer("ChgOffDate"), # explicit end on simulation
        # DisbursementDate, DisbursementGross , BalanceGross ( 14 valeurs seulement différentes de zéro  ?)
        # MIS_Status : Excluded because it is Y !!!
        # ChgOffPrinGr : excluded
        (StandardScaler(), ["GrAppv"]), 
        #(StandardScaler(),["SBA_Appv"]),  
        remainder='passthrough'))

In [7]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from catboost import CatBoostClassifier, Pool
from scipy.stats import uniform, randint

# CatBoost prend en charge les données catégorielles nativement
categorical_features = ['State','NAICS', 'RevLineCr']

y = explanable_X['MIS_Status']
X = explanable_X.drop('MIS_Status', axis=1)

# Split en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Préparation des données pour CatBoost (en spécifiant les caractéristiques catégorielles)
# Convertir les données d'entraînement en un objet Pool pour indiquer les variables catégorielles (si nécessaire)
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features)

test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_features)


In [ ]:
#durée estimée : 20 min (100 fits de 12 secondes), réelle : 

# Initialize CatBoostClassifier
cat_boost_model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=6, cat_features=categorical_features)

pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', cat_boost_model)])

# Define the hyperparameters grid for RandomizedSearch
param_dist = {
    'iterations': randint(100, 200),       # Number of boosting iterations
    'depth': randint(5, 9),              # Depth of the trees
    'learning_rate': uniform(0.2, 0.4),  # Learning rate
    'l2_leaf_reg': uniform(3, 12),        # L2 regularization coefficient
    'border_count': randint(32, 255),     # Number of splits for categorical features
    'bagging_temperature': uniform(0, 1), # Controls the randomness in bagging
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(cat_boost_model, param_distributions=param_dist, 
                                   n_iter=20, scoring='precision', 
                                   cv=5, verbose=1, n_jobs=-1)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Meilleurs paramètres trouvés : ", random_search.best_params_)
print("Meilleur score Precision : ", random_search.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0:	learn: 0.4612104	total: 366ms	remaining: 1m 4s
1:	learn: 0.3688600	total: 892ms	remaining: 1m 17s
0:	learn: 0.4044363	total: 554ms	remaining: 1m 27s
0:	learn: 0.4622139	total: 744ms	remaining: 2m 10s
0:	learn: 0.4049025	total: 939ms	remaining: 2m 28s
0:	learn: 0.4924629	total: 849ms	remaining: 1m 55s
2:	learn: 0.3213470	total: 1.36s	remaining: 1m 18s
0:	learn: 0.4046656	total: 649ms	remaining: 1m 42s
0:	learn: 0.4608848	total: 1.18s	remaining: 3m 25s
0:	learn: 0.4613718	total: 1.01s	remaining: 2m 56s
0:	learn: 0.4609667	total: 874ms	remaining: 2m 32s
3:	learn: 0.2900102	total: 1.9s	remaining: 1m 21s
1:	learn: 0.3138916	total: 1.54s	remaining: 2m
1:	learn: 0.3660644	total: 1.69s	remaining: 2m 27s
1:	learn: 0.3928129	total: 1.81s	remaining: 2m 2s
2:	learn: 0.2618077	total: 2.13s	remaining: 1m 51s
0:	learn: 0.4040212	total: 1.22s	remaining: 3m 13s
4:	learn: 0.2688865	total: 2.69s	remaining: 1m 31s
1:	learn: 0.3144270	total: 

In [ ]:
#durée estimée : 45 min (225 fits de 12 sec), réelle : - on est jamais arrivé à la fin du gridsearch sans erreur
# avec une seule valeur : 5 * 12 sec =>

#Meilleur score AUC :  0.9771748465313973 avec one_hot_encoder
# param_dist = {
#     'iterations': [191],       # Number of boosting iterations
#     'depth': [7],              # Depth of the trees
#     'learning_rate': [0.2621537355139263],  # Learning rate
#     'l2_leaf_reg': [9.620670562880655],        # L2 regularization coefficient
#     'bagging_temperature': [0.09440670531484197], # Controls the randomness in bagging
# }

#Meilleur score AUC : 0.9773692063410346
# param_dist = {
#     'iterations': [173], 
#     'depth': [8], 
#     'learning_rate':[0.2628238092699549],
#     'l2_leaf_reg': [4.642981495447009], 
#     'bagging_temperature': [0.6728806036024608], 
#     'border_count': [106]
# }

#Meilleur score Precision :  0.9616689295201551
bm_iterations = 184
bm_depth = 6
bm_learning_rate = 0.28119761253249864
bm_l2_leaf_reg = 7.32060164065551
bm_bagging_temperature = 0.22998810110951384
bm_border_count = 240

best_params_random = random_search.best_params_
if random_search.best_score_ > 0.9616689295201551 :
    bm_iterations = best_params_random['iterations']
    bm_depth = best_params_random['depth'] 
    bm_learning_rate = best_params_random['learning_rate']
    bm_l2_leaf_reg = best_params_random['l2_leaf_reg']
    bm_bagging_temperature = best_params_random['bagging_temperature']
    bm_border_count = best_params_random['border_count']

# Initialize CatBoostClassifier
cat_boost_model = CatBoostClassifier(iterations = bm_iterations, learning_rate=bm_learning_rate, depth=bm_depth, cat_features=categorical_features)

pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', cat_boost_model)])

# Utilisation dans un gridsearch
param_grid = {
    'iterations': [bm_iterations],
    'depth': [bm_depth],
    'learning_rate': [bm_learning_rate],
    'l2_leaf_reg': [bm_l2_leaf_reg], 
    'bagging_temperature': [bm_bagging_temperature], 
    'border_count': [bm_border_count],
}

# Créer le GridSearchCV avec le param_grid
grid_search = GridSearchCV(estimator=cat_boost_model, param_grid=param_grid, 
                           scoring='precision', cv=5, n_jobs=-1, verbose=1)

# Ajuster GridSearchCV
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score
print("Meilleurs paramètres trouvés par GridSearch : ", grid_search.best_params_)
print("Meilleur score precision de GridSearch : ", grid_search.best_score_)


In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import MIS_Status_reference as mis_ref

num_ech = y_test.shape[0]
print(f"Nombre d'échantillons du test : {num_ech}")

test_score = grid_search.score(X_test, y_test)
#test_score = random_search.score(X_test, y_test)
print(f"Test score : {test_score * 100:.2f}%")

# Prédictions sur le jeu de test

#y_pred = random_search.predict(X_test)
y_pred = grid_search.predict(X_test)


df_y_test = pd.DataFrame(y_test)
#df_y_test["MIS_Status"] = df_y_test["MIS_Status"].apply(lambda x : mis_ref.convert_MIS_Status_to_int(x) )

df_y_pred = pd.DataFrame(y_pred, columns=["MIS_Status"])
#df_y_pred["MIS_Status"] = df_y_pred["MIS_Status"].apply(lambda x : mis_ref.convert_MIS_Status_to_int(x) )

#ligne 1 : pred_value = 0
conf_mat_TN = 0
conf_mat_FN = 0

#ligne 2 : pred_value = 1
conf_mat_FP = 0
conf_mat_TP = 0

for index in range(num_ech) :
    test_value = df_y_test.iloc[index, 0]
    pred_value = df_y_pred.iloc[index, 0]
    if pred_value == 0 :
        if test_value == 0 : conf_mat_TN +=1
        else : conf_mat_FP += 1
    else :
        if test_value == 0: conf_mat_FN += 1
        else : conf_mat_TP += 1

check_recall = conf_mat_TP / (conf_mat_FN + conf_mat_TP)
check_precision = conf_mat_TP / (conf_mat_FP+ conf_mat_TP)

# Évaluation du modèle
accuracy = accuracy_score(df_y_test, df_y_pred)
precision = precision_score(df_y_test, df_y_pred)
recall = recall_score(df_y_test, df_y_pred)
macro_f1 = f1_score(df_y_test, df_y_pred, average='macro')
conf_matrix = confusion_matrix(df_y_test, df_y_pred)

# Prédire sur l'ensemble de test
#y_pred_proba = random_search.predict_proba(X_test)[:, 1]  # Probabilités pour la classe positive
y_pred_proba = grid_search.predict_proba(X_test)[:, 1]  # Probabilités pour la classe positive

# Calculer l'AUC sur l'ensemble de test
auc = roc_auc_score(y_test, y_pred_proba)

# Afficher les résultats
print(f"AUC : {100*auc:.2f}%")
print(f"Accuracy : {100*accuracy:.2f}%")
print(f"Precision : {100*precision :.2f}% (check={100*check_precision:.2f}%, FN={conf_mat_FN})")
print(f"Recall : {100*recall:.2f}% (check={100*check_recall:.2f}%, FP={conf_mat_FP})")
print(f"Macro F1 Score : {100*macro_f1:.2f}%")
print("Matrice de confusion :")
print(conf_matrix)


# Affichage avec Seaborn (optionnel)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['pred=0', 'pred=1'], yticklabels=['test=0', 'test=1'])
plt.xlabel('y_pred')
plt.ylabel('y_test')
plt.title('Matrice de Confusion')
plt.show()

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))
